In [ ]:
import board
import busio
i2c = busio.I2C(board.SCL, board.SDA) #,  frequency = 100000)
from adafruit_bus_device.i2c_device import I2CDevice
device = I2CDevice(i2c, 0x50)
with device:
    device.write(b'\x00\x00') #bytes([0x00]))
    result = bytearray(7)
    device.readinto(result)
    #i2c.readfrom_into(0x42, result)
    byte_string = result
    
    print ('[{}]'.format(', '.join(hex(x) for x in result)))
    
#hex(i2c.scan()[0])

In [4]:
import board
import busio
i2c = busio.I2C(board.SCL, board.SDA,  frequency = 50000)
i2c.scan()

[32]

In [ ]:
from pyftdi.ftdi import Ftdi
Ftdi.show_devices()

In [1]:
import board
print(dir(board))


['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'D4', 'D5', 'D6', 'D7', 'I2C', 'MISO', 'MOSI', 'SCK', 'SCL', 'SCLK', 'SDA', 'SPI', '__blinka__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__repo__', '__spec__', '__version__', 'ap_board', 'board_id', 'detector', 'pin', 'sys']


In [2]:
import os
print(os.environ["BLINKA_FT232H"])

1


In [ ]:
import board
import busio
i2c = busio.I2C(board.SCL, board.SDA,  frequency = 50000)
while True :
    a_list = i2c.scan()
    print ('[{}]'.format(', '.join(hex(x) for x in a_list)))

In [ ]:
#qui marche
import time
import board
import busio
i2c = busio.I2C(board.SCL, board.SDA)

while not i2c.try_lock():
    pass
try:
    temph = bytearray(1)
    templ = bytearray(1)
    result1 = bytearray(1)
    result2 = bytearray(1)
    i2c.writeto(0x68,b'\x6B\x00')    # Send instruction to start sensing
    time.sleep(0.05)                 # Small wait to give sensor time
    i2c.writeto(0x68,b'\x23\x40')    # Send instruction to start sensing
    time.sleep(0.05)                 # Small wait to give sensor time
    #i2c.writeto(0x68, bytes([0x6B]))
    #i2c.writeto(0x68, bytes([0x0F]))
    #result = bytearray(2)
    i2c.writeto(0x68,b'\x6B')
    i2c.readfrom_into(0x68, result1)
    i2c.writeto(0x68,b'\x23')
    i2c.readfrom_into(0x68, result2)
    i2c.writeto(0x68,b'\x41')
    i2c.readfrom_into(0x68, temph)
    i2c.writeto(0x68,b'\x42')
    i2c.readfrom_into(0x68, templ)
finally:
    i2c.unlock()
#print ('[{}]'.format(', '.join(bin(x) for x in result1)))
#print ('[{}]'.format(', '.join(bin(x) for x in result2)))
print ('[{}]'.format(', '.join(bin(x) for x in temph)))
print ('[{}]'.format(', '.join(bin(x) for x in templ)))


temph_bin = int.from_bytes(temph, byteorder='big')
templ_bin = int.from_bytes(templ, byteorder='big')

temperature = (temph_bin << 8) | templ_bin
print(temperature)

if (temperature >= 0x8000):
    print((0xFFFF - temperature))
    temperature = -((0xFFFF - temperature) + 1)
else:
    temperature = temperature

C = temperature/340.0 + 36.53
print (C)

In [ ]:
#Qui marche
import time
import board
import busio
i2c = busio.I2C(board.SCL, board.SDA)

while not i2c.try_lock():
    pass
try:
    result = bytearray(16)
    i2c.writeto(0x50,b'\x00\x01\x56\x57\x5B')    # Send instruction to start sensing
    time.sleep(0.001)                 # Small wait to give sensor time

    i2c.writeto(0x50,b'\x00\x00')
    #time.sleep(0.002) 
    i2c.readfrom_into(0x50, result)
finally:
    i2c.unlock()
print ('[{}]'.format(', '.join(hex(x) for x in result)))

In [ ]:
# SPDX-FileCopyrightText: 2021 ladyada for Adafruit Industries
# SPDX-License-Identifier: MIT

from board import SCL, SDA
from busio import I2C
from adafruit_bus_device.i2c_device import I2CDevice
from adafruit_register.i2c_struct import UnaryStruct

DEVICE_ADDRESS = 0x68  # device address of PCA9685 board
A_DEVICE_REGISTER_1 = 0x6B  # Configuration register

class DeviceControl:  # pylint: disable-msg=too-few-public-methods
    def __init__(self, i2c):
        self.i2c_device = i2c  # self.i2c_device required by UnaryStruct class

    register1 = UnaryStruct(A_DEVICE_REGISTER_1, "<B")  # 8-bit number



# The follow is for I2C communications
comm_port = I2C(SCL, SDA)
device = I2CDevice(comm_port, DEVICE_ADDRESS)
registers = DeviceControl(device)

# set the bits in the device
registers.register1 = 0xFF

# display the device values for the bits
print("register 1: {:02X}".format(registers.register1))

# toggle the bits
registers.register1 = 0b01111111

# display the device values for the bits
print("register 1: {:02X}".format(registers.register1))

In [ ]:
print("register 1: {:02X}".format(registers.register1))

In [ ]:
import board
import busio
 
REGISTERS = (0, 0x76)  # Range of registers to read, from the first up to (but
                      # not including!) the second value.
 
REGISTER_SIZE = 1     # Number of bytes to read from each register.
 
# Initialize and lock the I2C bus.
i2c = busio.I2C(board.SCL, board.SDA)
while not i2c.try_lock():
    pass
 
# Find the first I2C device available.
devices = i2c.scan()
while len(devices) < 1:
    devices = i2c.scan()
device = devices[0]
print('Found device with address: {}'.format(hex(device)))
 
# Scan all the registers and read their byte values.
result = bytearray(REGISTER_SIZE)
for register in range(*REGISTERS):
    try:
        i2c.writeto(device, bytes([register]))
        i2c.readfrom_into(device, result)
    except OSError:
        continue  # Ignore registers that don't exist!
    print('Address {0}: {1}'.format(hex(register), ' '.join([hex(x) for x in result])))
 
# Unlock the I2C bus when finished.  Ideally put this in a try-finally!
i2c.unlock()

In [ ]:
# SPDX-FileCopyrightText: 2021 ladyada for Adafruit Industries
# SPDX-License-Identifier: MIT

import time
import board
import busio
import adafruit_mpu6050

i2c = busio.I2C(board.SCL, board.SDA)
mpu = adafruit_mpu6050.MPU6050(i2c)

while True:
    print("Acceleration: X:%.2f, Y: %.2f, Z: %.2f m/s^2" % (mpu.acceleration))
    print("Gyro X:%.2f, Y: %.2f, Z: %.2f degrees/s" % (mpu.gyro))
    print("Temperature: %.2f C" % mpu.temperature)
    print("")
    time.sleep(1)


In [ ]:
#DS1302
import time
import board
import busio
i2c = busio.I2C(board.SCL, board.SDA,  frequency = 1000000)
from adafruit_bus_device.i2c_device import I2CDevice
device = I2CDevice(i2c, 0x51)
with device :
    device.write(bytes([0x005]))
    #device.write(bytes([0x0]))
    #device.write(bytes([0x5]))
    result = bytearray(1)
    device.readinto(result)
    #print(result)
    print ('[{}]'.format(', '.join(hex(x) for x in result)))
    time.sleep(0.1)
    #device.write(bytes([0x00]))

In [ ]:
from adafruit_blinka.microcontroller.ftdi_mpsse.mpsse.i2c import I2C as _I2C
from adafruit_blinka.agnostic import board_id, detector
import adafruit_platformdetect.constants.boards as ap_board
import adafruit_platformdetect.constants.chips as ap_chip
from adafruit_blinka import Enum, Lockable, agnostic
from adafruit_blinka.agnostic import board_id, detector
detector.board.ftdi_ft232h
from adafruit_blinka.microcontroller.ftdi_mpsse.ft232h import pin
#from microcontroller.pin import i2cPorts
#_i2c.scan()
pin.SDA

In [ ]:
with device :
    device.write(bytes([0x00]))
    #device.write(bytes([0x00]))
    #device.write(bytes([0x05]))
    result = bytearray(1)
    device.readinto(result)
    #i2c.readfrom_into(0x42, result)
    #print(hex(result[0]))
    print(result)
    print ('[{}]'.format(', '.join(hex(x) for x in result)))

In [ ]:
device.readinto(result)
#i2c.readfrom_into(0x42, result)
#print(hex(result[0]))
print(result)
print ('[{}]'.format(', '.join(hex(x) for x in result)))

In [ ]:
# SPDX-FileCopyrightText: Copyright (c) 2021 Tim Cocks for Adafruit Industries
#
# SPDX-License-Identifier: Unlicense

import board
import adafruit_24lc32 #NACK from slave

i2c = board.I2C()  # uses board.SCL and board.SDA
# i2c = board.STEMMA_I2C()  # For using the built-in STEMMA QT connector on a microcontroller
eeprom = adafruit_24lc32.EEPROM_I2C(i2c)

print("length: {}".format(len(eeprom)))

# eeprom[0] = 4
# print(eeprom[0])

# eeprom[0:4] = [9, 3, 8, 1]
# print(eeprom[0:4])

In [ ]:
eeprom[0] = 4
print(eeprom[0])



In [ ]:
print(eeprom[0])

In [ ]:
print("length: {}".format(len(eeprom)))

In [ ]:
# SPDX-FileCopyrightText: 2021 Kattni Rembor for Adafruit Industries
# SPDX-License-Identifier: MIT
"""CircuitPython I2C Device Address Scan"""
import time
import board

i2c = board.I2C()  # uses board.SCL and board.SDA
# i2c = board.STEMMA_I2C()  # For using the built-in STEMMA QT connector on a microcontroller

# To create I2C bus on specific pins
# import busio
# i2c = busio.I2C(board.GP1, board.GP0)    # Pi Pico RP2040

while not i2c.try_lock():
    pass

try:
    while True:
        print(
            "I2C addresses found:",
            [hex(device_address) for device_address in i2c.scan()],
        )
        time.sleep(2)

finally:  # unlock the i2c bus when ctrl-c'ing out of the loop
    i2c.unlock()